CSV 로드

In [5]:
import pandas as pd 

df = pd.read_csv("feedback_log.csv")
df

,질문,답변번호,별점,피드백,인공지능 답변
0,헬로웅,1,3,몰라.\n,헬로! 어떻게 도와드릴까요? 궁금한 점이나 필요한 정보가 있으시다면 언제든지 말씀해...
1,저기요,1,2,몰라ㅏㅏㅏㅏㅏ,안녕하세요! LG AI Research의 EXAONE 모델입니다. 도움이 필요하시다...
2,안녕,1,3,잘가,안녕하세요! 😊 How can I assist you today? Feel free...


In [ ]:
# streamlit run app.py
import streamlit as st
from llama_index.core.llms import ChatMessage
from llama_index.core.memory import ChatMemoryBuffer
from llama_index.llms.ollama import Ollama
import pandas as pd, os, time, logging

logging.basicConfig(level=logging.INFO)
CSV_PATH = "feedback_log.csv"

# 세션 초기화
if "messages" not in st.session_state:
    st.session_state.messages = []
if "memory" not in st.session_state:
    st.session_state.memory = ChatMemoryBuffer.from_defaults(token_limit=2048)
if "feedback_states" not in st.session_state:
    st.session_state.feedback_states = {}
if "just_saved" not in st.session_state:
    st.session_state.just_saved = None  # 마지막으로 피드백 저장된 답변 번호

# CSV 파일 준비
if not os.path.exists(CSV_PATH):
    pd.DataFrame(columns=["질문", "별점", "피드백", "인공지능 답변"]).to_csv(CSV_PATH, index=False)

def append_feedback(rating, fb, question, answer):
    """질문 포함해서 피드백 저장"""

    # 좌우 공백 제거
    try:
        fb = str(fb).strip() 
    except:
        pass

    df = pd.DataFrame([{
        "질문": question,
        "별점": rating,
        "피드백": fb,
        "인공지능 답변": answer
    }])
    df.to_csv(CSV_PATH, mode="a", header=False, index=False)

def stream_chat(model, messages):
    llm = Ollama(model=model, request_timeout=180.0)
    resp = llm.stream_chat(messages)
    response, placeholder = "", st.empty()
    for r in resp:
        response += r.delta
        placeholder.write(response)
    return response

def main():
    st.title("LLM 모델과 채팅하기 (피드백 안정형)")
    model = st.sidebar.selectbox("모델 선택", ["exaone3.5:2.4b"])

    if st.sidebar.button("초기화"):
        st.session_state.messages = []
        st.session_state.memory = ChatMemoryBuffer.from_defaults(token_limit=2048)
        st.session_state.feedback_states = {}
        st.session_state.just_saved = None
        st.success("모든 대화 및 피드백이 초기화되었습니다.")

    # === 대화 렌더링 ===
    answer_idx = 0
    for i, msg in enumerate(st.session_state.messages):
        with st.chat_message(msg["role"]):
            st.write(msg["content"])
            if msg["role"] == "assistant":
                answer_idx += 1
                if answer_idx not in st.session_state.feedback_states:
                    st.session_state.feedback_states[answer_idx] = {
                        "rating": None, "feedback": "", "done": False
                    }
                fs = st.session_state.feedback_states[answer_idx]

                # ✅ 방금 저장된 답변은 바로 완료 메시지로 대체
                if st.session_state.just_saved == answer_idx:
                    fs["done"] = True
                    st.session_state.just_saved = None

                # === 사용자 질문 추출 ===
                # assistant 메시지 직전 user 메시지를 찾아 연결
                question = ""
                if i > 0 and st.session_state.messages[i-1]["role"] == "user":
                    question = st.session_state.messages[i-1]["content"]

                if not fs["done"]:
                    st.markdown("---")
                    st.markdown("**이 답변은 얼마나 만족스러웠나요?**")
                    cols = st.columns(5)
                    for s, col in enumerate(cols, start=1):
                        if col.button(f"⭐ {s}", key=f"star_{answer_idx}_{s}"):
                            fs["rating"] = s

                    if fs["rating"]:
                        st.markdown(f"선택한 별점: {fs['rating']} ⭐")
                        fs["feedback"] = st.text_area("피드백을 남겨주세요.",
                                                      key=f"fb_{answer_idx}",
                                                      value=fs["feedback"])
                        if st.button("피드백 저장", key=f"save_{answer_idx}"):
                            append_feedback(fs["rating"], fs["feedback"], question, msg["content"])
                            fs["done"] = True
                            st.session_state.feedback_states[answer_idx] = fs
                            st.session_state.just_saved = answer_idx  # 다음 렌더링에서 즉시 완료 표시
                            st.rerun()

                else:
                    st.caption("✅ 이 답변은 이미 피드백 완료되었습니다.")

    # === 입력 영역 ===
    prompt = st.chat_input("질문을 입력하세요")
    if prompt:
        with st.chat_message("user"):
            st.write(prompt)
        st.session_state.messages.append({"role": "user", "content": prompt})
        st.session_state.memory.put(ChatMessage(role="user", content=prompt))

        with st.chat_message("assistant"):
            start = time.time()
            with st.spinner("응답 생성 중..."):
                try:
                    history = st.session_state.memory.get_all()
                    response = stream_chat(model, history)
                    st.write(response)
                    st.caption(f"⏱ {time.time() - start:.2f}초")
                    st.session_state.messages.append({"role": "assistant", "content": response})
                    st.session_state.memory.put(ChatMessage(role="assistant", content=response))
                    st.rerun()
                except Exception as e:
                    st.error(str(e))

if __name__ == "__main__":
    main()


In [ ]:
# 그동안의 대화 저장
import streamlit as st
from llama_index.core.llms import ChatMessage
from llama_index.core.memory import ChatMemoryBuffer
from llama_index.llms.ollama import Ollama
import pandas as pd, os, time, logging, json

logging.basicConfig(level=logging.INFO)
CSV_PATH = "feedback_log.csv"
SAVE_FILE = "conversations.json"

# === 세션 초기화 ===
if "conversations" not in st.session_state:
    st.session_state.conversations = {}  # {session_name: [messages]}
if "current_chat" not in st.session_state:
    st.session_state.current_chat = None
if "feedback_states" not in st.session_state:
    st.session_state.feedback_states = {}
if "memory" not in st.session_state:
    st.session_state.memory = ChatMemoryBuffer.from_defaults(token_limit=2048)

# === JSON 저장/불러오기 ===
def save_conversations():
    with open(SAVE_FILE, "w", encoding="utf-8") as f:
        json.dump(st.session_state.conversations, f, ensure_ascii=False, indent=2)

def load_conversations():
    if os.path.exists(SAVE_FILE):
        with open(SAVE_FILE, "r", encoding="utf-8") as f:
            st.session_state.conversations = json.load(f)

# === CSV 파일 준비 ===
if not os.path.exists(CSV_PATH):
    pd.DataFrame(columns=["세션", "답변번호", "별점", "피드백", "인공지능 답변"]).to_csv(CSV_PATH, index=False)

def append_feedback(session, answer_num, rating, fb, content):
    df = pd.DataFrame([{
        "세션": session,
        "답변번호": answer_num,
        "별점": rating,
        "피드백": fb,
        "인공지능 답변": content
    }])
    df.to_csv(CSV_PATH, mode="a", header=False, index=False)

def stream_chat(model, messages):
    llm = Ollama(model=model, request_timeout=180.0)
    resp = llm.stream_chat(messages)
    response, placeholder = "", st.empty()
    for r in resp:
        response += r.delta
        placeholder.write(response)
    return response

# === 메인 ===
def main():
    st.title("💬 LLM 모델과 채팅하기 (대화 세션 + 피드백 포함)")

    # === 왼쪽 사이드바 ===
    st.sidebar.header("📂 대화 관리")

    # 이전 세션 불러오기
    if st.sidebar.button("🔄 저장된 대화 불러오기"):
        load_conversations()
        st.success("저장된 대화를 불러왔습니다.")

    # 새 대화 시작
    if st.sidebar.button("➕ 새 대화"):
        new_name = time.strftime("%Y-%m-%d_%H-%M-%S")
        st.session_state.conversations[new_name] = []
        st.session_state.current_chat = new_name
        st.session_state.feedback_states[new_name] = {}
        st.success(f"새 대화 시작: {new_name}")

    # 대화 목록
    if st.session_state.conversations:
        st.sidebar.subheader("🗂️ 기존 대화 목록")
        for name in st.session_state.conversations.keys():
            if st.sidebar.button(name):
                st.session_state.current_chat = name

    model = st.sidebar.selectbox("모델 선택", ["exaone3.5:2.4b"])
    st.sidebar.button("💾 전체 저장", on_click=save_conversations)

    # === 현재 대화 표시 ===
    if not st.session_state.current_chat:
        st.info("왼쪽 사이드바에서 새 대화를 시작하거나 불러오세요.")
        return

    session_name = st.session_state.current_chat
    messages = st.session_state.conversations[session_name]
    feedback_state = st.session_state.feedback_states.setdefault(session_name, {})

    st.subheader(f"💭 현재 대화: {session_name}")

    answer_idx = 0
    for msg in messages:
        with st.chat_message(msg["role"]):
            st.write(msg["content"])
            if msg["role"] == "assistant":
                answer_idx += 1
                fs = feedback_state.setdefault(answer_idx, {"rating": None, "feedback": "", "done": False})
                if not fs["done"]:
                    st.markdown("---")
                    st.markdown("**이 답변은 얼마나 만족스러웠나요?**")
                    cols = st.columns(5)
                    for s, col in enumerate(cols, start=1):
                        if col.button(f"⭐ {s}", key=f"{session_name}_star_{answer_idx}_{s}"):
                            fs["rating"] = s
                    if fs["rating"]:
                        st.markdown(f"선택한 별점: {fs['rating']} ⭐")
                        fs["feedback"] = st.text_area("피드백을 남겨주세요.",
                                                      key=f"{session_name}_fb_{answer_idx}",
                                                      value=fs["feedback"])
                        if st.button("피드백 저장", key=f"{session_name}_save_{answer_idx}"):
                            append_feedback(session_name, answer_idx, fs["rating"], fs["feedback"], msg["content"])
                            fs["done"] = True
                            feedback_state[answer_idx] = fs
                            st.success("피드백이 저장되었습니다! 🙌")
                            st.rerun()
                else:
                    st.caption("✅ 이 답변은 이미 피드백 완료되었습니다.")

    # === 사용자 입력 ===
    if prompt := st.chat_input("질문을 입력하세요"):
        # 사용자 메시지 추가
        messages.append({"role": "user", "content": prompt})
        st.session_state.memory.put(ChatMessage(role="user", content=prompt))
        with st.chat_message("user"):
            st.write(prompt)

        # LLM 응답 스트리밍
        with st.chat_message("assistant"):
            start = time.time()
            with st.spinner("응답 생성 중..."):
                try:
                    history = st.session_state.memory.get_all()
                    response = stream_chat(model, history)
                    st.write(response)
                    st.caption(f"⏱ {time.time() - start:.2f}초")
                    messages.append({"role": "assistant", "content": response})
                    st.session_state.memory.put(ChatMessage(role="assistant", content=response))
                except Exception as e:
                    st.error(str(e))

if __name__ == "__main__":
    main()
